## Imports

In [ ]:
import os
import math

import numpy as np
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
sns.set()

In [ ]:
import mixtureofconcave as subm
import determinantal as logsubm
import plottingtools

## Plottingtools

In [ ]:
def plot_objective_values(eval_objectives, eval_ground, setcolor, groundcolor=None):
    """
    As we greedily build out the set,
    we plot the value of the objective f(S_t)
    at every stage t
    :param eval_objectives: f(S_t) for 0 \leq t < k
    :param eval_ground: f(V)
    :param setcolor: color to plot f(S_t) with
    :param groundcolor: color to plot f(V) with
    """
    k = len(eval_objectives)
    plt.plot(np.arange(k), eval_objectives, "o", c=setcolor, label="$f(S_t)$")
    if groundcolor is not None:
        plt.plot(np.arange(k), [eval_ground,]*k, "--", c=groundcolor, label="$f(V)$")
    plt.xlabel("$|S_t|$")
    plt.legend(loc=2)

In [ ]:
def plot_membership_histogram(
    memberships, budgets,
    S,
    setcolor, groundcolor,
    budgetlabel="budget", setlabel="selection"
):
    """ Given a membership assignment matrix (n x p) and optionally group budgets (min or max),
        A selection of indices S of size k < n
        Plot the selection's group-wise distribution
        :param memberships: n rows (members), p columns (0 or 1 membership bools)
        :param budgets: indices in [n], length k
        :param setcolor: color to plot f(S_t) with
        :param groundcolor: color to plot f(V) with
    """
    
    [n, p] = memberships.shape
    k = len(S)
    
    Vcounts = np.sum(memberships, axis=0)
    Scounts = np.sum(memberships[S], axis=0)
    if budgetlabel == "quotas":
        unmet = np.where(Scounts < budgets)[0]
    elif budgetlabel == "capacities":
        unmet = np.where(Scounts < capacities)[0]
    else:
        unmet = None
    
    plt.bar(np.arange(p), Vcounts, color=groundcolor)  # all members
    plt.bar(np.arange(p), Scounts, color=setcolor, label=setlabel)  # selected members
    plt.scatter(np.arange(p), budgets, c="white", s=16, zorder=8, label=budgetlabel)  # all budgets
    if unmet is not None:
        plt.scatter(unmet, budgets[unmet], c="red", s=2, zorder=10, label="unmet "+budgetlabel)  # unmet budgets
    plt.legend()


---

## Submodular Objective

In [ ]:
def concave_function(x):
    """
    \phi(x)
    """
    # options: modA**0.5, np.log(1+modA), (1-np.exp(-modA)), modA/(1+modA)
    return x**0.5


In [ ]:
class submodular_oracle:
    
    def __init__(self, concave_function, weights, X):
        """
        :param concave_function: function \phi
        :param weights: for linear combination of \phi(\sum_{i \in A} X_{ij})) (length m)
        :param X: feature matrix (n x m)
        """
        self.concave_function = concave_function
        self.weights = weights
        self.X = X
        self.n = X.shape[0]
        self.m = X.shape[1]
        self.set = []
        self.modular_values = np.zeros(m)
        self.current_objective = 0
    
    def add_element(self, idx):
        self.modular_values += X[idx,:]
        self.current_objective = np.dot(
            self.weights, self.concave_function(self.modular_values)
        )
        self.set.append(idx)
    
    def compute_gain(self, idx):
        new_modular_values = self.modular_values + X[idx,:]
        return np.dot(
            self.weights, self.concave_function(new_modular_values)
        ) - self.current_objective

    def compute_set_value(self, indices):
        modular_values = np.sum(X[indices, :], axis=0)
        return np.dot(
            self.weights, self.concave_function(modular_values)
        )

In [ ]:
def greedy_vanilla(k):
    """ 
    For a given ground set, a feature matrix and mixture weights which define the submodular objective,
    Returns the greedy selection and step-wise objective values
    Over the addition of k items
    :param k: total capacity
    """
    
    oracle = submodular_oracle(concave_function, weights, X)
    
    V = np.arange(n).tolist()
    objs = []
    objs.append(0)
    
    for kk in range(k):
        maxgain = -100
        greedyv = np.random.choice(V)
        
        gains = [oracle.compute_gain(ii) for ii in V]
        greedy_winner = V[np.argmax(gains)]
        
        # move winner from V to A
        V.remove(greedy_winner)
        oracle.add_element(greedy_winner)
        objs.append(oracle.current_objective)
    
    return oracle.set, objs


In [ ]:
def greedy_quota(memberships, unfilled_quotas, k, verbose=False):
    """
    :param memberships: n rows (members), p columns (0 or 1 membership bools)
    :param unfilled_quotas: indices in [n], numpy array of length p
    :param k: total capacity
    """
    oracle = submodular_oracle(concave_function, weights, X)

    assert memberships.shape[0] == X.shape[0]
    assert memberships.shape[1] == len(unfilled_quotas)
    [n, p] = memberships.shape
    
    V = np.arange(n).tolist()
    objs = []
    objs.append(0)
    
    for jj in range(p):
        if np.sum(memberships[:,jj]) < unfilled_quotas[jj]:
            print("Not enough members in group {}, infeasible problem.".format(jj))
            return None, None
    
    """ Quota-filling stage """
    
    unfilled_groups = np.where(np.array(unfilled_quotas)>0)[0]
    V_quota_candidates = [ii for ii in V if np.sum(memberships[ii][unfilled_groups]) > 0]
    
    while np.sum(unfilled_quotas) > 0:
        
        unfilled_groups = np.where(unfilled_quotas>0)[0]
        V_quota_candidates = [ii for ii in V if np.sum(memberships[ii][unfilled_groups]) > 0]
    
        maxgain = -100
        greedyv = np.random.choice(V)
        
        gains = [oracle.compute_gain(ii) for ii in V]
        greedy_winner = V[np.argmax(gains)]
        
        # move winner from V to A
        V.remove(greedy_winner)
        oracle.add_element(greedy_winner)
        objs.append(oracle.current_objective)
        
        # decrement unfilled quotas
        unfilled_quotas = unfilled_quotas - memberships[greedy_winner]
        unfilled_quotas.clip(0, k)
        
        if verbose:
            print("unfilled_quotas = ", unfilled_quotas)
    
    """ Regular greedy stage """
    
    for kk in range(k - len(oracle.set)):
        maxgain = -100
        greedyv = np.random.choice(V)
        
        gains = [oracle.compute_gain(ii) for ii in V]
        greedy_winner = V[np.argmax(gains)]
        
        # move winner from V to A
        V.remove(greedy_winner)
        oracle.add_element(greedy_winner)
        objs.append(oracle.current_objective)
    
    return oracle.set, objs


### Test

In [ ]:
## Setup

n = 50
m = 200
k = 10
p = 3

np.random.seed(0)

X = np.random.random((n, m))
weights = np.random.random(m); weights = weights/np.max(weights)

disjoint_memberships = np.random.choice([0, 1], size=(n, p))
disjoint_quotas = np.array([2, 2, 2])

In [ ]:
## Ground set eval

oracle = submodular_oracle(concave_function, weights, X)
ground = oracle.compute_set_value(np.arange(n))

In [ ]:
## Submod greedy

S, objectives = greedy_vanilla(k)
plt.figure(figsize=(5,5))
plot_objective_values(objectives, ground, "darkcyan", "dimgrey")

In [ ]:
## Quota greedy (Disjoint)

S, objectives = greedy_quota(disjoint_memberships, disjoint_quotas, k)
plt.figure(figsize=(5,5))
plot_objective_values(objectives, ground, "darkorange", "dimgrey")

---

## Aadhi Quotoba mag Vithoba

Compare:<br>
Feasibility, output quality, computational complexity.

---

<p style="background-color:#ff9933">
Constrained Submodular Max with Disjoint Membership Quota
</p>

In [ ]:
n_samples = 1000
m_features = 2
k_budget = 100

np.random.seed(0)
X = np.random.random((n_samples, m_features))

np.random.seed(1)
mixw = np.random.random(m_features); mixw = mixw/np.max(mixw)

In [ ]:
p_groups = 20
memcolors = np.array(["#11{:02X}dd".format(pp) for pp in np.arange(0, 256, 256//p_groups)])

In [ ]:
#%% Random group assignment

np.random.seed(2)
Memvec = np.eye(p_groups)[np.random.choice(p_groups, n_samples)].astype(int)

#print([len(np.argwhere(Memvec[:,jj])) for jj in range(p_groups)])
print(np.sum(Memvec, axis=0))

#%%Clustering-based group assignment

# kmeans = KMeans(n_clusters=p_groups, random_state=0).fit(X)
# Memvec = np.eye(p_groups)[kmeans.labels_].astype(int)

#%% Viz

# vizpca_DMQ(X, Memvec, memcolors, [], "orange", "Group assignment")

In [ ]:
#%% Uniform quota

quo = np.ones(p_groups)*3

#%% Proportionate quota

# quo = np.sum(Memvec, axis=0)*k_budget//n_samples - 3

In [ ]:
S_s, objectives_s = subm.greedygains_submod(None, X, mixw, k_budget)
S_sq, objectives_sq = subm.greedyDMquota_submod(None, X, mixw, Memvec, quo, k_budget)
S_r = np.random.choice(n_samples, k_budget)
objective_sr = subm.submodgains(X, np.sum(X[S_r,:], axis=0), 0, None, mixw)

print("Objective without quota", objectives_s[-1], "\n", S_s)
print("Objective with quota", objectives_sq[-1], "\n", S_sq)
print("Objective of random selection", objective_sr, "\n", S_r)

plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plot_membership_histogram(Memvec, quo, S_s, "darkorange", "darkcyan", budgetlabel="quotas", setlabel="SPP")
plt.subplot(1,3,2)
plot_membership_histogram(Memvec, quo, S_sq, "yellowgreen", "darkcyan", budgetlabel="quotas", setlabel="SPP-DMQ")
plt.subplot(1,3,3)
plot_membership_histogram(Memvec, quo, S_r, "indianred", "darkcyan", budgetlabel="quotas", setlabel="Random")

In [ ]:
# S_sq, objectives_sq = greedyDMquota_submod(None, X, mixw, Memvec, quo, k_budget, True)

= = = = 

Can we show that there is some benefit to adding _good_ elements _after_ we have met
fairness (here, diversity?! if they are geometrically correlated...) constraints.

We don't want to confuse _good_ with _diverse_. I mean we do want to pick the good ones among the diverse tho...<br>
Is it like -- the group that is picked last gets more quality canditades chosen? and the one that goes first gets filled with token shit??

= = = = 

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
vizpca(X, "darkcyan", S_s, "darkorange", "SPP")
plt.subplot(1,2,2)
vizpca_DMQ(X, Memvec, memcolors, S_sq, "yellowgreen", "SPP-DMQ")

In [ ]:
viztsne_IMQ(X, Memvec, ["lightskyblue", "darkcyan"], S_sq, "yellowgreen", "SPP-DMQ", 30)

---

<p style="background-color:#ff99cc">
Constrained Submodular Max with Intersecting Membership Quota
</p>

In [ ]:
n_samples = 1000
m_features = 50
k_budget = 100

np.random.seed(0)
X = np.random.random((n_samples, m_features))

np.random.seed(1)
mixw = np.random.random(m_features); mixw = mixw/np.max(mixw)

In [ ]:
p_groups = 20
memcolors = np.array(["#11{:02X}dd".format(pp) for pp in np.arange(0, 256, 256//p_groups)])

In [ ]:
#%% Random group assignment

# np.random.seed(2)
# group_probabs = np.random.uniform(0.05, 0.45, p_groups)
# Memvec = np.zeros((n_samples, p_groups)).astype(int)
# for jj in range(p_groups):
#     Memvec[:,jj] = np.random.binomial(1, group_probabs[jj], n_samples)

# print(np.sum(Memvec, axis=0))

#%%Clustering-based group assignment

kmeans = KMeans(n_clusters=p_groups+3, random_state=0).fit(X)
Memvec = np.eye(p_groups+3, p_groups)[kmeans.labels_].astype(int)

np.random.seed(2)
group_probabs = np.random.uniform(0.05, 0.2, p_groups)
for jj in range(p_groups):
    Memvec[:,jj] = np.maximum(Memvec[:,jj], np.random.binomial(1, group_probabs[jj], n_samples))

#%% Viz

# vizpca_DMQ(X, Memvec, memcolors, [], "orange", "Group assignment")

In [ ]:
#%% Uniform quota

# quo = np.ones(p_groups)*2

#%% Proportionate quota

quo = np.sum(Memvec, axis=0)*k_budget//n_samples - 2

In [ ]:
S_s, objectives_s = subm.greedygains_submod(None, X, mixw, k_budget)
S_sq, objectives_sq = subm.greedyIMquota_submod(None, X, mixw, Memvec, quo, k_budget)
S_r = np.random.choice(n_samples, k_budget)
objective_sr = subm.submodgains(X, np.sum(X[S_r,:], axis=0), 0, None, mixw)

print("Objective without quota", objectives_s[-1], "\n", S_s)
print("Objective with quota", objectives_sq[-1], "\n", S_sq)
print("Objective of random selection", objective_sr, "\n", S_r)

plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plottingtools.vizbalance_MQ(np.arange(n_samples), Memvec, "darkcyan", quo, S_s, "darkorange", "SPP")
plt.subplot(1,3,2)
plottingtools.vizbalance_MQ(np.arange(n_samples), Memvec, "darkcyan", quo, S_sq, "yellowgreen", "SPP-DMQ")
plt.subplot(1,3,3)
plottingtools.vizbalance_MQ(np.arange(n_samples), Memvec, "darkcyan", quo, S_r, "indianred", "Random")

---

<p style="background-color:#ccff66">
Constrained Submodular Max with Feature Quota
</p>